In [24]:
import os
import pickle

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
with open(os.path.join('../intermediate/output/', 'val.pkl'), 'rb') as f:
    target = (pickle.load(f) != 0).reshape(768, -1)

In [25]:
file_list

['val_dn_31.pkl',
 'val_tn_31.pkl',
 'val_tn_41.pkl',
 'val_unet_27.pkl',
 'val_unet_33.pkl',
 'val_unet_38.pkl',
 'val_unet_40.pkl',
 'val_unet_41.pkl']

In [3]:
file_list = ['val_dn_31.pkl', 'val_tn_31.pkl',
             'val_tn_41.pkl', 'val_unet_27.pkl',
             'val_unet_33.pkl', 'val_unet_38.pkl', 'val_unet_40.pkl',
             'val_unet_41.pkl']
val_list = []
for file in file_list:
    with open(os.path.join('../intermediate/output/', file), 'rb') as f:
        val_list.append(pickle.load(f))

In [12]:
def dice_score(probs):
    m1    = probs.reshape(768, -1)
    score = (2*(m1*target).sum(axis=1))/(m1.sum(axis=1)+target.sum(axis=1))
    
    return score.sum()/768

In [5]:
def loss(weight=None, threshold=127.5):
    if weight is None:
        weight = [1.] * len(val_list)
     
    pred = 0.
    for i in range(len(val_list)):
        if weight[i] == 0:
            continue
        pred += weight[i] * val_list[i]
    pred = pred / sum(weight)
    pred = pred > threshold
    return dice_score(pred)

In [58]:
def vote_loss(weight=None, threshold=127.5):
    if weight is None:
        weight = [1.] * len(val_list)
     
    pred = 0.
    for i in range(len(val_list)):
        if weight[i] == 0:
            continue
        pred += weight[i] * (val_list[i] > threshold)
#    pred = pred / sum(weight)
    pred = pred > 2.5
    return dice_score(pred)

In [61]:
weight=[1., 1., 1., 0., 1., 1., 0., 1.]
vote_loss(weight, threshold=127)

0.9971631102040126

In [66]:
weight=[1., 1., 1., 1., 0., 1., 0., 1.]
vote_loss(weight, threshold=128)

0.99716889115338692

In [63]:
weight=[1., 1., 1., 1., 1., 0., 0., 1.]
vote_loss(weight, threshold=127)

0.9971598368133141

In [60]:
weight=[1., 0., 1., 1., 1., 1., 0., 1.]
vote_loss(weight, threshold=127)

0.99715858297492543

In [40]:
weight=[0., 0., 1., 1., 1., 1., 0., 1.]
vote_loss(weight, threshold=127)

0.99716014605501091

In [33]:
weight=[0., 1., 1., 1., 1., 1., 0., 2.]
vote_loss(weight)

0.99714061943529408

In [28]:
weight=[0., 3., 6., 2., 4., 6., 0., 11.]
vote_loss(weight)

0.99670701666381978

In [ ]:
weight=[3., 3., 6., 2., 4., 6., 0., 11.]
loss(weight)

In [29]:
weight=[0., 3., 6., 2., 4., 6., 0., 11.]
loss(weight)

0.99720060927407383

In [36]:
weight=[0., 3., 6., 2., 4., 6., 0., 13.]
loss(weight, 127)

0.99720058926852928

In [39]:
weight=[3., 3., 6., 2., 4., 6., 0., 11.]
loss(weight, 125)

0.99720587645403358